## A simple regression training using LightGBM through Fairing

In [1]:
import fairing

# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)
BASE_IMAGE = 'gcr.io/{}/lightgbm:latest'.format(GCP_PROJECT)

## Build a base image for LightGBM
You need to build the docker image only once and that can be reused in future runs.
Approximate time for build and push: 10mins

In [2]:
!docker build . -t {BASE_IMAGE}
!docker push {BASE_IMAGE}

Sending build context to Docker daemon  88.58kB
Step 1/9 : FROM python:3.7
^C
The push refers to repository [gcr.io/caip-dexter-bugbash/lightgbm]
An image does not exist locally with the tag: gcr.io/caip-dexter-bugbash/lightgbm


## Launch a LightGBM train task

In [3]:
import fairing
from fairing.frameworks import lightgbm

In [4]:
# Creating a bucket for copying the trained model. 
# You can set gcs_bucket variable to an existing bucket name if that is desired.
gcs_bucket = "gs://{}-fairing".format(GCP_PROJECT)
!gsutil mb {gcs_bucket}

Creating gs://caip-dexter-bugbash-fairing/...
ServiceException: 409 Bucket caip-dexter-bugbash-fairing already exists.


In [5]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'metric_freq': 1,
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    "n_estimators": 10,
    "is_training_metric": "true",
    "valid_data": "gs://fairing-lightgbm/regression-example/regression.test",
    "train_data": "gs://fairing-lightgbm/regression-example/regression.train",
    'verbose': 1,
    "model_output": "{}/lightgbm/example/model.txt".format(gcs_bucket)
}

In [6]:
lightgbm.execute(config=params,
      docker_registry=DOCKER_REGISTRY,
      base_image=BASE_IMAGE)

Building image using Append builder...
Creating docker context: /tmp/fairing_context_swxv4j48
Loading Docker credentials for repository 'gcr.io/caip-dexter-bugbash/lightgbm:latest'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Image successfully built in 1.4019325920000085s.
Pushing image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:F6051542...
Loading Docker credentials for repository 'gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:F6051542'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Uploading gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:F6051542
Layer sha256:c5e155d5a1d130a7f8a3e24cee0d9e1349bff13f90ec6a941478e558fde53c14 exists, skipping
Layer sha256:425d7b2a5bcc7e8df5041d66655cd6173c16799afefb1645ca0e0b3fa8a1043d exists, skipping
Layer sha256:a73086e5ef516ba8f40f945c6efe6d0d2f6b3ad04bfd29d914e67cf0f27aeffc exists, skipping
Layer sha256:1051e

Copying gs://fairing-lightgbm/regression-example/regression.train.weight...
/ [1 files][131.6 KiB/131.6 KiB]                                                
Operation completed over 1 objects/131.6 KiB.
Copying gs://fairing-lightgbm/regression-example/regression.train...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.
Copying gs://fairing-lightgbm/regression-example/regression.test...
/ [1 files][ 85.7 KiB/ 85.7 KiB]                                                
Operation completed over 1 objects/85.7 KiB.
All files are copied!
[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Loading weights...
[LightGBM] [Info] Finished loading data in 0.144989 seconds
[LightGBM] [Info] Total Bins 6143
[LightGBM] [Info] Number of data: 7000, number of used features: 28
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] Start training from score 0.527422
[L

Cleaning up job fairing-job-vhhsx...


## Let's look at the trained model

In [ ]:
url = params['model_output']
!gsutil cp {url} /tmp/model.txt
!head /tmp/model.txt